# Setup

In [ ]:
%pip install spuco

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 52.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.4/127.4 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.8/88.8 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.2/126.2 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.8/78.8 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.9/56.9 kB 4.9 MB/s eta 0:00:00
  Created wheel for grad-cam: filename=grad_cam-1.5.4-py3-none-any.whl size=39588 sha256=7c14dac77e049e5c559b43fefffb69424319f0e4f9c057dc66d45f6027f1d780
  Stored in directory: /root/.cache/pip/wheels/50/b0/82/1f97b5348c7fe9f0ce0ba18497202cafa5dec4562bd5292680
Successfully built grad-cam


In [ ]:
import torch

device = torch.device("cpu")

In [ ]:
from spuco.utils import set_seed

set_seed(0)

# Dataset Creation

In [ ]:
from spuco.datasets import SpuCoMNIST, SpuriousFeatureDifficulty
import torchvision.transforms as T

classes = [[0, 1], [2, 3], [4, 5], [6, 7], [8, 9]]
difficulty = SpuriousFeatureDifficulty.MAGNITUDE_LARGE

trainset = SpuCoMNIST(
    root="/data/mnist/",
    spurious_feature_difficulty=difficulty,
    spurious_correlation_strength=0.995,
    classes=classes,
    split="train",
    label_noise=0.001
)
trainset.initialize()

testset = SpuCoMNIST(
    root="/data/mnist/",
    spurious_feature_difficulty=difficulty,
    classes=classes,
    split="test"
)
testset.initialize()


Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 9.91M/9.91M [00:00<00:00, 11.6MB/s]


Extracting /data/mnist/MNIST/raw/train-images-idx3-ubyte.gz to /data/mnist/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 28.9k/28.9k [00:00<00:00, 349kB/s]


Extracting /data/mnist/MNIST/raw/train-labels-idx1-ubyte.gz to /data/mnist/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 1.65M/1.65M [00:00<00:00, 3.20MB/s]


Extracting /data/mnist/MNIST/raw/t10k-images-idx3-ubyte.gz to /data/mnist/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 4.54k/4.54k [00:00<00:00, 4.73MB/s]


Extracting /data/mnist/MNIST/raw/t10k-labels-idx1-ubyte.gz to /data/mnist/MNIST/raw



In [ ]:
from spuco.datasets import SpuCoMNIST, SpuriousFeatureDifficulty

classes = [[0, 1], [2, 3], [4, 5], [6, 7], [8, 9]]

valset = SpuCoMNIST(
    root="/data/mnist/",
    spurious_feature_difficulty=difficulty,
    classes=classes,
    split="val"
)
valset.initialize()


In [ ]:
T.ToPILImage()(trainset[1000][0]).resize((28,28))

In [ ]:
from spuco.models import model_factory
from spuco.robust_train import ERM
from torch.optim import SGD

# Train using JTT

## Group Inference

In [ ]:
#jtt params
jtt_epochs = 2
upsample_param = 100

In [ ]:
from spuco.utils import Trainer

infer_model = model_factory("lenet", trainset[0][0].shape, trainset.num_classes).to(device)
gi_trainer = Trainer(
    model=infer_model,
    trainset=trainset,
    batch_size=64,
    optimizer=SGD(infer_model.parameters(), lr=1e-2, momentum=0.9, nesterov=True),
    device=device,
    verbose=True
)
gi_trainer.train(jtt_epochs)

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
Epoch 0:   1%|          | 7/751 [00:00<01:01, 12.02batch/s, accuracy=31.25%, loss=1.6]  

 | Epoch 0 | Loss: 1.6159617900848389 | Accuracy: 15.625%
 | Epoch 0 | Loss: 1.6190482378005981 | Accuracy: 23.4375%
 | Epoch 0 | Loss: 1.6121280193328857 | Accuracy: 23.4375%
 | Epoch 0 | Loss: 1.6065869331359863 | Accuracy: 18.75%
 | Epoch 0 | Loss: 1.614766001701355 | Accuracy: 15.625%
 | Epoch 0 | Loss: 1.614539623260498 | Accuracy: 14.0625%
 | Epoch 0 | Loss: 1.6029881238937378 | Accuracy: 21.875%
 | Epoch 0 | Loss: 1.6199455261230469 | Accuracy: 31.25%
 | Epoch 0 | Loss: 1.6045913696289062 | Accuracy: 25.0%
 | Epoch 0 | Loss: 1.60679292678833 | Accuracy: 28.125%
 | Epoch 0 | Loss: 1.6028460264205933 | Accuracy: 23.4375%
 | Epoch 0 | Loss: 1.5993651151657104 | Accuracy: 31.25%


Epoch 0:   3%|▎         | 20/751 [00:01<00:24, 30.17batch/s, accuracy=35.9375%, loss=1.6]

 | Epoch 0 | Loss: 1.5930511951446533 | Accuracy: 25.0%
 | Epoch 0 | Loss: 1.5978997945785522 | Accuracy: 28.125%
 | Epoch 0 | Loss: 1.6100412607192993 | Accuracy: 21.875%
 | Epoch 0 | Loss: 1.6001620292663574 | Accuracy: 25.0%
 | Epoch 0 | Loss: 1.5925073623657227 | Accuracy: 21.875%
 | Epoch 0 | Loss: 1.5983284711837769 | Accuracy: 12.5%
 | Epoch 0 | Loss: 1.595309853553772 | Accuracy: 20.3125%
 | Epoch 0 | Loss: 1.594624638557434 | Accuracy: 18.75%
 | Epoch 0 | Loss: 1.5994746685028076 | Accuracy: 21.875%
 | Epoch 0 | Loss: 1.5901758670806885 | Accuracy: 25.0%
 | Epoch 0 | Loss: 1.5893046855926514 | Accuracy: 34.375%
 | Epoch 0 | Loss: 1.600570559501648 | Accuracy: 35.9375%


Epoch 0:   5%|▍         | 34/751 [00:01<00:15, 44.88batch/s, accuracy=84.375%, loss=1.5]  

...

Epoch 1:  99%|█████████▉| 746/751 [00:17<00:00, 35.22batch/s, accuracy=100.0%, loss=0.00396]

 | Epoch 1 | Loss: 0.0038287893403321505 | Accuracy: 100.0%
 | Epoch 1 | Loss: 0.004097089171409607 | Accuracy: 100.0%
 | Epoch 1 | Loss: 0.12054327875375748 | Accuracy: 98.4375%
 | Epoch 1 | Loss: 0.004650173243135214 | Accuracy: 100.0%
 | Epoch 1 | Loss: 0.004161848220974207 | Accuracy: 100.0%
 | Epoch 1 | Loss: 0.004385319072753191 | Accuracy: 100.0%
 | Epoch 1 | Loss: 0.1507030427455902 | Accuracy: 98.4375%
 | Epoch 1 | Loss: 0.003959109541028738 | Accuracy: 100.0%


Epoch 1: 100%|█████████▉| 750/751 [00:17<00:00, 35.43batch/s, accuracy=100.0%, loss=0.00381]

 | Epoch 1 | Loss: 0.1249382421374321 | Accuracy: 98.4375%
 | Epoch 1 | Loss: 0.003811272792518139 | Accuracy: 100.0%


Epoch 1: 100%|██████████| 751/751 [00:17<00:00, 42.71batch/s, accuracy=100.0%, loss=0.00381]


In [ ]:
output = gi_trainer.get_trainset_outputs()
max_output = torch.max(output, dim=1)
pred_labels = max_output.indices
pred_labels = pred_labels.tolist()
actual_labels = trainset.labels

Getting model outputs: 100%|██████████| 751/751 [00:07<00:00, 104.06batch/s]


In [ ]:
from spuco.group_inference import JTTInference

jtt = JTTInference(pred_labels, actual_labels)
gi = jtt.infer_groups()
nominal_upsample = len(gi[(0, 0)])/len(gi[(0, 1)])
print(nominal_upsample)
print(len(gi[(0, 0)]))
print(len(gi[(0, 1)]))

163.96219931271477
47713
291


In [ ]:
upsampled_indices = [i for i in gi[(0, 1)] for x in range(upsample_param)]
upsampled_indices.extend(gi[(0, 0)])
len(upsampled_indices)

76813

## Robust Training

In [ ]:
from spuco.robust_train import CustomSampleERM

robust_model = model_factory("lenet", trainset[0][0].shape, trainset.num_classes).to(device)
rt_erm = CustomSampleERM(
    model=robust_model,
    trainset=trainset,
    batch_size=64,
    optimizer=SGD(robust_model.parameters(), lr=1e-2, momentum=0.9, nesterov=True),
    num_epochs=3,
    indices=upsampled_indices,
    device=device,
    verbose=True
)
rt_erm.train()

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
Epoch 0:   1%|          | 7/1201 [00:00<00:45, 26.52batch/s, accuracy=20.3125%, loss=1.62]

 | Epoch 0 | Loss: 1.6116225719451904 | Accuracy: 20.3125%
 | Epoch 0 | Loss: 1.6139423847198486 | Accuracy: 18.75%
 | Epoch 0 | Loss: 1.6238937377929688 | Accuracy: 17.1875%
 | Epoch 0 | Loss: 1.620105266571045 | Accuracy: 10.9375%
 | Epoch 0 | Loss: 1.6177009344100952 | Accuracy: 23.4375%
 | Epoch 0 | Loss: 1.610742449760437 | Accuracy: 26.5625%
 | Epoch 0 | Loss: 1.606016755104065 | Accuracy: 23.4375%
 | Epoch 0 | Loss: 1.6140230894088745 | Accuracy: 29.6875%
 | Epoch 0 | Loss: 1.621118426322937 | Accuracy: 20.3125%
 | Epoch 0 | Loss: 1.6115514039993286 | Accuracy: 23.4375%
 | Epoch 0 | Loss: 1.6064388751983643 | Accuracy: 25.0%
 | Epoch 0 | Loss: 1.6186648607254028 | Accuracy: 20.3125%


Epoch 0:   2%|▏         | 20/1201 [00:00<00:25, 46.26batch/s, accuracy=40.625%, loss=1.6] 

 | Epoch 0 | Loss: 1.6085240840911865 | Accuracy: 18.75%
 | Epoch 0 | Loss: 1.6164621114730835 | Accuracy: 17.1875%
 | Epoch 0 | Loss: 1.608508825302124 | Accuracy: 23.4375%
 | Epoch 0 | Loss: 1.6061453819274902 | Accuracy: 23.4375%
 | Epoch 0 | Loss: 1.608201265335083 | Accuracy: 17.1875%
 | Epoch 0 | Loss: 1.6102447509765625 | Accuracy: 15.625%
 | Epoch 0 | Loss: 1.6060258150100708 | Accuracy: 23.4375%
 | Epoch 0 | Loss: 1.6045933961868286 | Accuracy: 18.75%
 | Epoch 0 | Loss: 1.604992151260376 | Accuracy: 25.0%
 | Epoch 0 | Loss: 1.604671835899353 | Accuracy: 34.375%
 | Epoch 0 | Loss: 1.6043440103530884 | Accuracy: 31.25%
 | Epoch 0 | Loss: 1.6003762483596802 | Accuracy: 35.9375%


Epoch 0:   3%|▎         | 34/1201 [00:00<00:20, 57.23batch/s, accuracy=35.9375%, loss=1.59]

...

Epoch 2:  99%|█████████▉| 1191/1201 [00:27<00:00, 36.91batch/s, accuracy=100.0%, loss=0.00025]

 | Epoch 2 | Loss: 0.00040516743320040405 | Accuracy: 100.0%
 | Epoch 2 | Loss: 0.008305086754262447 | Accuracy: 100.0%
 | Epoch 2 | Loss: 0.0029633280355483294 | Accuracy: 100.0%
 | Epoch 2 | Loss: 0.0003682622336782515 | Accuracy: 100.0%
 | Epoch 2 | Loss: 0.0008280432666651905 | Accuracy: 100.0%
 | Epoch 2 | Loss: 0.023488763719797134 | Accuracy: 98.4375%
 | Epoch 2 | Loss: 0.043151263147592545 | Accuracy: 98.4375%
 | Epoch 2 | Loss: 0.00025030714459717274 | Accuracy: 100.0%


Epoch 2: 100%|█████████▉| 1196/1201 [00:27<00:00, 39.84batch/s, accuracy=100.0%, loss=0.00015]

 | Epoch 2 | Loss: 0.0008408439462073147 | Accuracy: 100.0%
 | Epoch 2 | Loss: 0.0013603329425677657 | Accuracy: 100.0%
 | Epoch 2 | Loss: 0.001005953294225037 | Accuracy: 100.0%
 | Epoch 2 | Loss: 0.001093021361157298 | Accuracy: 100.0%
 | Epoch 2 | Loss: 0.0015914911637082696 | Accuracy: 100.0%
 | Epoch 2 | Loss: 0.0020659524016082287 | Accuracy: 100.0%
 | Epoch 2 | Loss: 0.0008883465779945254 | Accuracy: 100.0%
 | Epoch 2 | Loss: 0.0017928136512637138 | Accuracy: 100.0%
 | Epoch 2 | Loss: 0.0012206768151372671 | Accuracy: 100.0%
 | Epoch 2 | Loss: 0.00015003902080934495 | Accuracy: 100.0%


Epoch 2: 100%|██████████| 1201/1201 [00:27<00:00, 43.58batch/s, accuracy=100.0%, loss=0.00015]


In [ ]:
from spuco.evaluate import Evaluator

rt_evaluator = Evaluator(
    testset=testset,
    group_partition=testset.group_partition,
    group_weights=trainset.group_weights,
    batch_size=64,
    model=robust_model,
    device=device,
    verbose=True
)
rt_evaluator.evaluate()

Evaluating group-wise accuracy:   4%|▍         | 1/25 [00:00<00:06,  3.44it/s]

Group (0, 0) Accuracy: 99.76359338061465


Evaluating group-wise accuracy:   8%|▊         | 2/25 [00:00<00:06,  3.52it/s]

Group (0, 1) Accuracy: 69.03073286052009


Evaluating group-wise accuracy:  12%|█▏        | 3/25 [00:00<00:06,  3.47it/s]

...

Evaluating group-wise accuracy:  96%|█████████▌| 24/25 [00:06<00:00,  3.42it/s]

Group (4, 3) Accuracy: 66.66666666666667


Evaluating group-wise accuracy: 100%|██████████| 25/25 [00:07<00:00,  3.46it/s]

Group (4, 4) Accuracy: 99.74747474747475


{(0, 0): 99.76359338061465,
 (0, 1): 69.03073286052009,
 (0, 2): 76.59574468085107,
 (0, 3): 57.4468085106383,
 (0, 4): 60.7565011820331,
 (1, 0): 67.97066014669926,
 (1, 1): 98.77750611246944,
 (1, 2): 72.79411764705883,
 (1, 3): 48.529411764705884,
 (1, 4): 41.42156862745098,
 (2, 0): 77.06666666666666,
 (2, 1): 62.666666666666664,
 (2, 2): 100.0,
 (2, 3): 63.2,
 (2, 4): 33.155080213903744,
 (3, 0): 76.63316582914572,
 (3, 1): 54.15617128463476,
 (3, 2): 77.32997481108312,
 (3, 3): 99.74811083123426,
 (3, 4): 65.23929471032746,
 (4, 0): 84.1309823677582,
 (4, 1): 30.478589420654913,
 (4, 2): 26.448362720403022,
 (4, 3): 66.66666666666667,
 (4, 4): 99.74747474747475}

In [ ]:
rt_evaluator.worst_group_accuracy

((4, 2), 26.448362720403022)

In [ ]:
rt_evaluator.average_accuracy

99.37059573457985

In [ ]:
rt_evaluator.evaluate_spurious_attribute_prediction()

19.84